# Optimizing Model Parameters

现在我们有了model和data，是时候通过依靠data更新model的参数来训练(train)、验证(validate)以及测试(test)我们的model了。训练一个model是一个反复(iterative)的过程;在每一次的迭代张红(一次epoch)，model会输出一个结果(guess about the output),计算它guess(loss)的error率，收集error的derivative，然后通过gradient descent去更新这些parameters让它们达到最优值。关于这个过程的更多细节，请过看这个视频[backpropagation from 3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8)

## 预备代码
下面的代码是在之前章节写过的代码。

2.Datasets & DataLoaders

4.Build Model

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor()
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

train_dataloader = DataLoader(training_data, batch_size=64)
test_dataloader = DataLoader(test_data, batch_size=64)

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork()

E:\Software\Anaconda3\envs\py38\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


## Hyperparameters

Hyperparameters是可调节的parameters，通过它让你可以控制model optimization的过程。不同的hyperparameter值可以影响model的训练和收敛点（convergence rate）（阅读关于更多的[hyperparameter tuning](https://pytorch.org/tutorials/beginner/hyperparameter_tuning_tutorial.html)）

我们在训练过程中定义了如下的hyperparameters：

* **Number of Epoch**-在dataset上迭代的次数
* **Batch Size**-在parameter更新前每次通过network的data sample数量
* **Learning rate**-在每次的batch\epoch更新模型parameter的幅度。值越小意味着更小的学习速率，值过大可能在训练过程中导致不可预料的事情。

In [4]:
learning_rate=1e-3
batch_size=64
epoch=5

## Optimization Loop

当我们设置好了hyperparameter后，我们可以在optimization循环中训练和更新我们的model，每次的循环中的一次迭代都叫做一次的**Epoch**。

每次Epoch包含两个重要的部分：

* **The Train Loop**-在training dataset上迭代，尽可能的收敛到parameter的最优点。
* **The Validation/Test Loop**-在test dataset上迭代来检验model的表现是否有得到提升。

我们简要的了解一下在training loop中的一些概念。

### Loss函数
我们没有训练过的network面对training data时候往往不能给出正确的答案。**Loss函数**就是度量通过network获得的结果和target value不同的程度，在训练的过程中，我们需要让loss函数的值能达到最低点。计算loss的方式一般是我们通过给定的input data值做出一个预测值，用它来比较data的label值。

常见的loss function包括回归任务的[nn.MSELoss](https://pytorch.org/docs/stable/generated/torch.nn.MSELoss.html#torch.nn.MSELoss)(Mean Square Error)和分类任务的[nn.NLLLoss](https://pytorch.org/docs/stable/generated/torch.nn.NLLLoss.html#torch.nn.NLLLoss)(Negative Log Likelihood).[nn.CrossEntropyLoss](https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss)将<kbd>nn.LogSoftmax</kbd>和<kbd>nn.NLLLoss</kbd>连接到了一起。

我们将output logits输入到<kbd>nn.CrossEntropyLoss</kbd>，通过这样将logits标准化然后计算prediction error。

In [5]:
# Initialize the Loss function
loss_fn=nn.CrossEntropyLoss()

### Optimizer
Optimization是调整model parameters在每一次的training中降低model error的过程。Optimization算法定义了这个过程的执行方式（在这个例子中我们使用Stochastic Gradient Descent）。所有的optimization逻辑都封装在<kbd>optimizer object</kbd>中，这个例子中我们使用SGD optimizer；此外，在Pytorch中还有很多不同的Optimizer可以使用，例如ADAM和RMSProp，这些Optimizer在不同种类的model和data中可能会表现的更好。

我们通过传入需要训练的model's parameter以及learning rate hyperparameter来初始化optimizer。


In [6]:
optimizer=torch.optim.SGD(model.parameters(),lr=learning_rate)

在training loop内部optimization过程一共分三步：
* 调用<kbd>optimizer.zero_grad()</kbd>重置model parameter的gradient。Gradient默认累加的；防止加两次，我们要在每次迭代的时候将它们清零。
* 调用 <kbd>loss.backwards()</kbd>方法反向传播prediction loss。Pytorch会存储每个parameter的loss的gradient。
* 得到gradient以后，我们调用 <kbd>optimizer.step()</kbd>函数通过收集backward pass过程中的gradient来调整parameter。

## 全部代码

我们定义<kbd>train_loop</kbd>来循环我们的optimization部分的代码，定义<kbd>test_loop</kbd>评估在test data上的model's performance。

In [9]:
def train_loop(dataloader,model,loss_fn,optimizer):
    size=len(dataloader.dataset)
    for batch,(X,y)in enumerate(dataloader):
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred,y)
        
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if batch%100 == 0:
            loss,current = loss.item(),batch*len(X)
            print(f"loss:{loss:>7f} [{current:>5d}/{size:>5d}]")

def test_loop(dataloader,model,loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    
    with torch.no_grad():
        for X,y in dataloader:
            pred=model(X)
            test_loss+=loss_fn(pred,y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

我们初始化loss函数和optimizer，然后将其加入到<kbd>train_loop</kbd>和<kbd>test_loop</kbd>。增加epoch次数并追踪model提升的效果。

In [10]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train_loop(train_dataloader, model, loss_fn, optimizer)
    test_loop(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss:2.300437 [    0/60000]
loss:2.296579 [ 6400/60000]
loss:2.283700 [12800/60000]
loss:2.284281 [19200/60000]
loss:2.293092 [25600/60000]
loss:2.278162 [32000/60000]
loss:2.271635 [38400/60000]
loss:2.258407 [44800/60000]
loss:2.256768 [51200/60000]
loss:2.267684 [57600/60000]
Test Error: 
 Accuracy: 33.8%, Avg loss: 2.246487 

Epoch 2
-------------------------------
loss:2.241510 [    0/60000]
loss:2.252903 [ 6400/60000]
loss:2.212900 [12800/60000]
loss:2.221671 [19200/60000]
loss:2.246316 [25600/60000]
loss:2.224112 [32000/60000]
loss:2.211269 [38400/60000]
loss:2.189314 [44800/60000]
loss:2.187185 [51200/60000]
loss:2.217227 [57600/60000]
Test Error: 
 Accuracy: 38.6%, Avg loss: 2.171180 

Epoch 3
-------------------------------
loss:2.164196 [    0/60000]
loss:2.187643 [ 6400/60000]
loss:2.106689 [12800/60000]
loss:2.122961 [19200/60000]
loss:2.181246 [25600/60000]
loss:2.150117 [32000/60000]
loss:2.123044 [38400/60000]
loss:2.091353 [44800

## 拓展阅读
* [Loss Functions](https://pytorch.org/docs/stable/nn.html#loss-functions)
* [torch.optim](https://pytorch.org/docs/stable/optim.html)
* [Warmstart Training a Model](https://pytorch.org/tutorials/recipes/recipes/warmstarting_model_using_parameters_from_a_different_model.html)